In [26]:
import numpy as np
import os
import torch
import torchvision
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from mpl_toolkits.mplot3d import axes3d
from  PIL import Image
from time import time
from torch import nn
from torchvision import models
from torchvision import transforms
from torchvision import datasets
from torch.utils.data.dataset import Dataset
from torchsummary import summary
import pylab
import random
import numpy 
import pandas as pd

from sklearn.decomposition import PCA
from sklearn import preprocessing

torch.cuda.is_available()

False

In [27]:
resnet18 = models.resnet18(pretrained=True)

In [28]:
resnet34 = models.resnet34(pretrained=True)

In [29]:
resnet50 = models.resnet50(pretrained=True)

In [30]:
resnet101 = models.resnet101(pretrained=True)

In [31]:
resnet152 = models.resnet152(pretrained=True)

In [32]:
# choose the model.
resnet = resnet50

In [33]:
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )])

In [34]:
def load_images_from_folder(folder, end, as_tensor = False):

    images = []
    files = folders = 0
    count= 0
    dirnames = [] 

    num_img = []

    for _, dirnames, filenames in os.walk(folder):

 

        if dirnames != []:
            subfolders = dirnames
        current_path = os.path.join(folder, subfolders[count])
        num_img.append(len(os.listdir(current_path)))
        for filename in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, filename))
            if img is not None:
                if as_tensor:
                    img = torch.from_numpy(img * 1.0)
                    images.append(img)
                else: 
                    images.append(img)

 

        count += 1
        
        if count % 10 == 0:
            print('number of folder done =', count, 'total number of images so far =', len(images))
            
        if count  == end:
            return images, num_img

    return images, num_img


def show_image(PIL_img):
    PIL_img.show()

In [35]:
path = "../uob_image_set"

dataset, num_img_list = load_images_from_folder(path, 10)

number of folder done = 10 total number of images so far = 42


In [36]:
print(len(dataset)), print(num_img_list), print(sum(num_img_list))

42
[5, 4, 4, 5, 4, 3, 4, 4, 5, 4]
42


(None, None, None)

In [37]:
list_input_tensor = []
list_input_batch = []
num_img = sum(num_img_list)
for item in range(num_img):
    input_tensor = preprocess(dataset[item])
    input_batch = input_tensor.unsqueeze(0)
    list_input_tensor.append(input_tensor)
    list_input_batch.append(input_batch)
    


In [38]:
for param in resnet.parameters():
    param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = resnet.fc.in_features
# resnet.fc = torch.nn.Linear(num_ftrs, 3)


In [57]:
for i in range(len(list_input_batch)):
    if torch.cuda.is_available():
        input_batch = list_input_batch[i].to('cuda')
        resnet.to('cuda')
    else:
        input_batch = list_input_batch[i].to('cpu')
        resnet.to('cpu')
        
    
    with torch.no_grad():
        output = resnet(input_batch)


In [60]:
output_array = []
for i in range(len(list_input_batch)):
    if torch.cuda.is_available():
        input_batch = list_input_batch[i].to('cuda')
        resnet.to('cuda')
    else:
        input_batch = list_input_batch[i].to('cpu')
        resnet.to('cpu')
        
    with torch.no_grad():
        output = resnet(input_batch)
        cup_tensor = output.cpu()
            
    output_array.append(cup_tensor[0].tolist())
#     output_array.append(np.asarray(cup_tensor[0]))
#     print(np.asarray(cup_tensor[0]))
output_array = np.asarray(output_array)


In [61]:

fig = plt.figure()
ax = plt.axes(projection="3d")

img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    ax.scatter(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,1], output_array[img_counter:img_counter + n,2],color=random_color,s=20)
    img_counter += n
plt.show()  

ax.set_xlabel('X Axes')
ax.set_ylabel('Y Axes')
ax.set_zlabel('Frame Axes')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:

fig = plt.figure()
ax = plt.axes(projection="3d")

img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    ax.scatter(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,1], output_array[img_counter:img_counter + n,2],color=random_color,s=20)
    img_counter += n
plt.show()  

ax.set_xlabel('X Axes')
ax.set_ylabel('Y Axes')
ax.set_zlabel('Frame Axes')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
# fig = plt.figure()
# img_counter = 0
# for n in num_img_list:
#     r = lambda: random.randint(0,255)
#     random_color = '#%02X%02X%02X' % (r(),r(),r())
#     plt.plot(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,1], 'o',color=random_color)
#     img_counter += n
# plt.show()            

In [64]:
# fig = plt.figure()
# img_counter = 0
# for n in num_img_list:
#     r = lambda: random.randint(0,255)
#     random_color = '#%02X%02X%02X' % (r(),r(),r())
#     plt.plot(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,2], 'o',color=random_color)
#     img_counter += n
# plt.show()            

In [65]:
# fig = plt.figure()
# img_counter = 0
# for n in num_img_list:
#     r = lambda: random.randint(0,255)
#     random_color = '#%02X%02X%02X' % (r(),r(),r())
#     plt.plot(output_array[img_counter:img_counter + n,1], output_array[img_counter:img_counter + n,2], 'o',color=random_color)
#     img_counter += n
# plt.show()            

In [66]:
scaled_data = preprocessing.scale(output_array)
pca = PCA()
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)

In [67]:
pca_data

array([[ 1.85456919e+00,  2.00794147e+00, -1.04878016e+01, ...,
        -1.89399473e+00, -3.25649939e+00, -3.66373598e-15],
       [-1.85095606e+00, -4.36659365e-01, -6.83563224e+00, ...,
        -1.98987830e+00, -8.03296473e-01,  2.11636264e-15],
       [-1.23923265e+01,  2.19866946e+00, -1.55074730e+01, ...,
         4.22722237e+00,  2.42296936e+00,  4.41313652e-15],
       ...,
       [-6.74673305e+00,  3.32856905e+00, -2.81968846e+00, ...,
        -3.37178579e+00,  9.81944645e-01, -4.81212292e-15],
       [-1.06639903e+01, -5.68263421e+00, -4.25676372e+00, ...,
         1.63490575e+00, -9.21283250e-01, -3.77475828e-15],
       [ 5.89668441e+01,  8.73805268e+00, -8.63622053e-01, ...,
         2.48468742e-01, -1.20127058e+00,  3.21964677e-15]])

In [68]:
per_var = np.round(pca.explained_variance_ratio_*100, decimals = 1)

In [69]:
per_var

array([21.4,  8.4,  6.2,  5.6,  4.5,  3.4,  3.4,  3. ,  2.7,  2.7,  2.4,
        2.2,  2.1,  1.9,  1.8,  1.8,  1.8,  1.7,  1.7,  1.5,  1.5,  1.4,
        1.3,  1.2,  1.2,  1.1,  1.1,  1. ,  1. ,  1. ,  0.9,  0.9,  0.8,
        0.8,  0.7,  0.7,  0.7,  0.6,  0.6,  0.6,  0.5,  0. ])

In [70]:
# plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label = labels)
labels = ['PC' +  str(x) for x in range(1,len(per_var)+1)]
plt.bar(x=range(1,len(per_var)+1), height=per_var)
plt.show()

In [73]:
%matplotlib widget


pca_df = pd.DataFrame(pca_data, columns=labels)

fig = plt.figure()
ax = plt.axes(projection="3d")

img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    plt.plot(pca_df.PC1[img_counter:img_counter + n], pca_df.PC2[img_counter:img_counter + n],pca_df.PC3[img_counter:img_counter + n], 'o',color=random_color)
    img_counter += n
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …